In [1]:
from decimal import Decimal # for filenames

import pdb #Equivalent of keyboard in MATLAB, just add "pdb.set_trace()"

import os
import sys

In [2]:
###############################################################################
#                       HyperParameters and RunOptions                        #
###############################################################################
class Hyperparameters:
    max_hidden_layers = 15 # For this architecture, need at least 2. One for the mapping to the feature space, one as a trainable hidden layer. EXCLUDES MAPPING BACK TO DATA SPACE
    num_hidden_nodes  = 100
    activation        = 'elu'
    classification_act= 'linear'
    regularization    = 0.0001
    manifold          = 0.001
    node_TOL          = 1e-4
    error_TOL         = 1e-4
    batch_size        = 50
    num_epochs        = 400
    
    num_networks      = 6
    

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
from Utilities.get_image_data import load_data
from Utilities.form_train_val_test_batches import form_train_val_test_batches
from Utilities.NN_FC_layerwise import FCLayerwise
from Utilities.NN_FC_layerwise_new import FCLayerwise_new
from Utilities.NETW import Final
from Utilities.Net import Final_Network
from Utilities.Net_new import Final_Network_ALGO_II
from Utilities.create_data import create_new
from Utilities.create_data_multiply import create_new_multiply
from Utilities.loss_and_accuracies import data_loss_classification, data_loss_regression
from Utilities.manifold_regularization import manifold_classification
from Utilities.manifold_regularization_new import manifold_classification_new
from Utilities.optimize_layerwise import optimize
from Utilities.optimize_step_II import optimize_step
from Utilities.additive_output import net_output 
from Utilities.plot_and_save_figures_layerwise import plot_fig
from joblib import Parallel, delayed
import multiprocessing
import random

In [4]:
class Hyperparameters_new:
    max_hidden_layers = 3 # For this architecture, need at least 2. One for the mapping to the feature space, one as a trainable hidden layer. EXCLUDES MAPPING BACK TO DATA SPACE
    num_hidden_nodes  = 10
    activation        = 'elu'
    classification_act= 'linear'
    regularization    = 0.000
    manifold          = 0.000
    node_TOL          = 1e-4
    error_TOL         = 1e-4
    batch_size        = 100
    num_epochs        = 2000

In [5]:
class RunOptions:
    def __init__(self):    
        #=== Choose Which GPU to Use ===#
        self.which_gpu = '1'
        
        #=== Use L_1 Regularization ===#
        self.use_L1 = 1
        
        #=== Choose Data Set ===#
        self.data_MNIST = 0
        self.data_CIFAR10 = 0 
        self.data_CIFAR100 = 0
        self.data_regression=1
        
        #=== Random Seed ===#
        self.random_seed = 1234


In [6]:
###############################################################################
#                                 File Paths                                  #
###############################################################################         
class FilePaths():    
    def __init__(self, hyperp, run_options):  
        #=== Declaring File Name Components ===# 
        self.NN_type = 'FC'
        if run_options.data_MNIST == 1:
            self.dataset = 'MNIST'
        if run_options.data_CIFAR10 == 1:
            self.dataset = 'CIFAR10'
        if run_options.data_CIFAR100 == 1:
            self.dataset = 'CIFAR100'
        if run_options.data_regression == 1:
            self.dataset = 'Abalone'
        if hyperp.regularization >= 1:
            hyperp.regularization = int(hyperp.regularization)
            regularization_string = str(hyperp.regularization)
        else:
            regularization_string = str(hyperp.regularization)
            regularization_string = 'pt' + regularization_string[2:]                        
        node_TOL_string = str('%.2e' %Decimal(hyperp.node_TOL))
        node_TOL_string = node_TOL_string[-1]
        error_TOL_string = str('%.2e' %Decimal(hyperp.error_TOL))
        error_TOL_string = error_TOL_string[-1]
        
        #=== File Name ===#
        if run_options.use_L1 == 0:
            self.filename = self.dataset + '_' + self.NN_type + '_mhl%d_hl%d_eTOL%s_b%d_e%d' %(hyperp.max_hidden_layers, hyperp.num_hidden_nodes, error_TOL_string, hyperp.batch_size, hyperp.num_epochs)
        else:
            self.filename = self.dataset + '_' + self.NN_type + '_L1_mhl%d_hl%d_r%s_nTOL%s_eTOL%s_b%d_e%d' %(hyperp.max_hidden_layers, hyperp.num_hidden_nodes, regularization_string, node_TOL_string, error_TOL_string, hyperp.batch_size, hyperp.num_epochs)

        #=== Saving Trained Neural Network and Tensorboard ===#
        #self.NN_savefile_directory = 'C:/Users/Chandradut/Desktop/Sparse training/Trained_NNs/' + self.filename # Since we need to save four different types of files to save a neural network model, we need to create a new folder for each model
        self.NN_savefile_directory =  self.filename
        self.NN_savefile_name = self.NN_savefile_directory + '/' + self.filename # The file path and name for the four files
        #self.tensorboard_directory = 'C:/Users/Chandradut/Desktop/Sparse training/Tensorboard/' + self.filename

###############################################################################



In [7]:
if __name__ == "__main__":     
    
    
#for trainable in range(2,10):
    
    trainable=2
    #=== Hyperparameters and Run Options ===#    
    hyperp = Hyperparameters()
    hyperp_new=Hyperparameters_new()
    run_options = RunOptions()
    

    #=== File Names ===#
    file_paths = FilePaths(hyperp, run_options)
    
    #=== Load Data ===#       
    data_train, labels_train,\
    data_test, labels_test,\
    data_input_shape, num_channels, label_dimensions, labels_train_manifold, data_train_manifold, data_val, labels_val\
    = load_data(file_paths.NN_type, file_paths.dataset, run_options.random_seed) 
    


    
    #for i in range(1,hyperp.num_networks):
    for i in range(1,2):
    #=== Initiate training ===#
        #trainer(hyperp, run_options, file_paths,i) 
        if i>1:
            trainable=2

    
            
        if trainable==2:
        
        
        
            #=== GPU Settings ===#
            os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
            os.environ["CUDA_VISIBLE_DEVICES"] = run_options.which_gpu
    
            #=== Neural Network ===#
            if run_options.use_L1 == 0:
                kernel_regularizer = None
                bias_regularizer = None  
            else:
                kernel_regularizer = tf.keras.regularizers.l1(hyperp.regularization)
                bias_regularizer = tf.keras.regularizers.l1(hyperp.regularization)
            
            
            multiply=0
        
            if multiply==0:

                data_train,new_label,labels_train=create_new(data_train, labels_train,hyperp,hyperp_new, run_options, data_input_shape, label_dimensions,i)
        
            if multiply==1:

                data_train,new_label,labels_train=create_new_multiply(data_train, labels_train,hyperp,hyperp_new, run_options, data_input_shape, label_dimensions,i)
     
        #=== Construct Validation Set and Batches ===# 
            data_and_labels_train, data_and_labels_val, data_and_labels_test,\
            num_data_train, num_data_val, num_data_test,\
            num_batches_train, num_batches_val, num_batches_test,data_and_labels_train_new\
            = form_train_val_test_batches(data_train, labels_train, \
                                      data_test, labels_test, \
                                      hyperp.batch_size, new_label, run_options.random_seed)
        
        
            # Creating validation data
            
            data_and_labels_val = tf.data.Dataset.from_tensor_slices((data_val, labels_val)).batch(hyperp.batch_size)
            num_data_val = len(list(data_and_labels_val))
            
    
        if i==1 and trainable==2:
            NN = FCLayerwise(hyperp, run_options, data_input_shape, label_dimensions,kernel_regularizer, bias_regularizer)    
            NN._set_inputs( data_train)
        if i>1:
            kernel_regularizer = None
            bias_regularizer = None
            NN = FCLayerwise_new(hyperp_new, run_options, data_input_shape, label_dimensions,kernel_regularizer, bias_regularizer) 
            NN._set_inputs( data_train)
    #=== Training ===#
    #                                 Training                                    #
###############################################################################
        if trainable>2:
            del NN
            NN = Final(hyperp, run_options, data_input_shape, label_dimensions,kernel_regularizer, bias_regularizer,trainable)   
            #NN._set_inputs(data_train)
            NN.load_weights("WEIGHTS"+'/'+"model_weights"+str(1)+str(trainable-1))
            #NN=tf.keras.models.load_model("WEIGHTS"+'/'+"model"+str(1)+str(trainable-1))
        


        if i==1:
            hyperp_n=hyperp
            optimize(hyperp,hyperp_n, run_options, file_paths, NN, data_loss_classification, data_loss_regression, data_and_labels_train, data_and_labels_val, data_and_labels_test, label_dimensions, num_batches_train,data_and_labels_train_new,manifold_classification,hyperp.batch_size,run_options.random_seed,num_data_train,i,data_input_shape,data_train,labels_train,multiply,trainable, labels_train_manifold, data_train_manifold)   
        
        if i>1:
            hyperp_n=Hyperparameters_new()
            optimize_step(hyperp,hyperp_n, run_options, file_paths, NN, data_loss_classification, data_loss_regression, data_and_labels_train, data_and_labels_val, data_and_labels_test, label_dimensions, num_batches_train,data_and_labels_train_new,manifold_classification_new,hyperp.batch_size,run_options.random_seed,num_data_train,i,data_input_shape,data_train,labels_train,multiply)   
        
        #NN.save("WEIGHTS"+'/'+"model"+str(1)+str(trainable))
        if not os.path.exists("WEIGHTS"):
            os.makedirs("WEIGHTS")
        NN.save_weights("WEIGHTS"+'/'+"model_weights"+str(1)+str(trainable))
        
       # if i==1:
          #  plot_fig(hyperp, run_options, file_paths,i,trainable+1)
            
      #  if i>1:
        #    plot_fig(hyperp_new, run_options, file_paths,i,3)
        
        

        
        
        
        
        
        
        
        

2022-06-28 18:23:19.141790: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-28 18:23:19.167426: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2593830000 Hz
2022-06-28 18:23:19.169162: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x56303ed5b0c0 executing computations on platform Host. Devices:
2022-06-28 18:23:19.169205: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
2022-06-28 18:23:19.171363: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


ListWrapper([10, 100, 100, 12])
Beginning Training
            Epoch 0            
Abalone_FC_L1_mhl15_hl100_rpt0001_nTOL4_eTOL4_b50_e400
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 1 batches of size 50:
Model: "fc_layerwise"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
W2 (Dense)                   multiple                  10100     
_________________________________________________________________
upsampling_layer (Dense)     multiple                  1100      
_________________________________________________________________
classification_layer (Dense) multiple                  1212      
Total params: 12,412
Trainable params: 12,412
Non-trainable params: 0
_________________________________________________________________

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X 

Time per Batch: 0.90
Time per Epoch: 1.00

Training Set: Loss: 9.195e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 1.066
Test Set: Loss: 0.000e+00, Accuracy: 0.985

Previous Layer Relative # of 0s: 0.0000000

            Epoch 16            
Abalone_FC_L1_mhl15_hl100_rpt0001_nTOL4_eTOL4_b50_e400
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 1 batches of size 50:
Time per Batch: 0.91
Time per Epoch: 1.00

Training Set: Loss: 9.168e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 1.060
Test Set: Loss: 0.000e+00, Accuracy: 0.978

Previous Layer Relative # of 0s: 0.0000000

            Epoch 17            
Abalone_FC_L1_mhl15_hl100_rpt0001_nTOL4_eTOL4_b50_e400
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 1 batches of size 50:
Time per Batch: 0.89
Time per Epoch: 0.98

Training Set: Loss: 9.142e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 1.061
Test Set: Loss: 0.000e+00, Accuracy: 0.981

Previous Layer Relative # of 0s: 0.000000

Time per Batch: 0.88
Time per Epoch: 0.99

Training Set: Loss: 8.592e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 1.015
Test Set: Loss: 0.000e+00, Accuracy: 0.931

Previous Layer Relative # of 0s: 0.0000000

            Epoch 35            
Abalone_FC_L1_mhl15_hl100_rpt0001_nTOL4_eTOL4_b50_e400
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 1 batches of size 50:
Time per Batch: 0.89
Time per Epoch: 1.00

Training Set: Loss: 8.574e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 1.003
Test Set: Loss: 0.000e+00, Accuracy: 0.935

Previous Layer Relative # of 0s: 0.0000000

            Epoch 36            
Abalone_FC_L1_mhl15_hl100_rpt0001_nTOL4_eTOL4_b50_e400
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 1 batches of size 50:
Time per Batch: 0.93
Time per Epoch: 1.04

Training Set: Loss: 8.536e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 1.010
Test Set: Loss: 0.000e+00, Accuracy: 0.927

Previous Layer Relative # of 0s: 0.000000

Time per Batch: 0.88
Time per Epoch: 0.98

Training Set: Loss: 8.101e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.961
Test Set: Loss: 0.000e+00, Accuracy: 0.902

Previous Layer Relative # of 0s: 0.0000000

            Epoch 54            
Abalone_FC_L1_mhl15_hl100_rpt0001_nTOL4_eTOL4_b50_e400
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 1 batches of size 50:
Time per Batch: 0.88
Time per Epoch: 0.97

Training Set: Loss: 8.068e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.959
Test Set: Loss: 0.000e+00, Accuracy: 0.886

Previous Layer Relative # of 0s: 0.0000000

            Epoch 55            
Abalone_FC_L1_mhl15_hl100_rpt0001_nTOL4_eTOL4_b50_e400
Trainable Hidden Layer Index: 2
GPU: 1

Optimizing 1 batches of size 50:
Time per Batch: 0.88
Time per Epoch: 0.98

Training Set: Loss: 8.055e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.957
Test Set: Loss: 0.000e+00, Accuracy: 0.899

Previous Layer Relative # of 0s: 0.000000

...

Time per Epoch: 2.78

Training Set: Loss: 2.885e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.487
Test Set: Loss: 0.000e+00, Accuracy: 0.459

Previous Layer Relative # of 0s: 0.6321782

            Epoch 226            
Abalone_FC_L1_mhl15_hl100_rpt0001_nTOL4_eTOL4_b50_e400
Trainable Hidden Layer Index: 11
GPU: 1

Optimizing 1 batches of size 50:
Time per Batch: 2.53
Time per Epoch: 2.78

Training Set: Loss: 2.877e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.472
Test Set: Loss: 0.000e+00, Accuracy: 0.423

Previous Layer Relative # of 0s: 0.6321782

            Epoch 227            
Abalone_FC_L1_mhl15_hl100_rpt0001_nTOL4_eTOL4_b50_e400
Trainable Hidden Layer Index: 11
GPU: 1

Optimizing 1 batches of size 50:
Time per Batch: 2.55
Time per Epoch: 2.81

Training Set: Loss: 2.885e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.487
Test Set: Loss: 0.000e+00, Accuracy: 0.459

Previous Layer Relative # of 0s: 0.6321782

            Ep

Time per Epoch: 2.82

Training Set: Loss: 2.884e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.487
Test Set: Loss: 0.000e+00, Accuracy: 0.458

Previous Layer Relative # of 0s: 0.6321782

            Epoch 244            
Abalone_FC_L1_mhl15_hl100_rpt0001_nTOL4_eTOL4_b50_e400
Trainable Hidden Layer Index: 11
GPU: 1

Optimizing 1 batches of size 50:
Time per Batch: 2.52
Time per Epoch: 2.77

Training Set: Loss: 2.876e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.472
Test Set: Loss: 0.000e+00, Accuracy: 0.422

Previous Layer Relative # of 0s: 0.6321782

            Epoch 245            
Abalone_FC_L1_mhl15_hl100_rpt0001_nTOL4_eTOL4_b50_e400
Trainable Hidden Layer Index: 11
GPU: 1

Optimizing 1 batches of size 50:
Time per Batch: 2.55
Time per Epoch: 2.80

Training Set: Loss: 2.884e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.487
Test Set: Loss: 0.000e+00, Accuracy: 0.458

Previous Layer Relative # of 0s: 0.6321782

            Ep

Time per Epoch: 2.81

Training Set: Loss: 2.883e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.487
Test Set: Loss: 0.000e+00, Accuracy: 0.458

Previous Layer Relative # of 0s: 0.6321782

            Epoch 262            
Abalone_FC_L1_mhl15_hl100_rpt0001_nTOL4_eTOL4_b50_e400
Trainable Hidden Layer Index: 11
GPU: 1

Optimizing 1 batches of size 50:
Time per Batch: 2.60
Time per Epoch: 2.86

Training Set: Loss: 2.875e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.471
Test Set: Loss: 0.000e+00, Accuracy: 0.422

Previous Layer Relative # of 0s: 0.6321782

            Epoch 263            
Abalone_FC_L1_mhl15_hl100_rpt0001_nTOL4_eTOL4_b50_e400
Trainable Hidden Layer Index: 11
GPU: 1

Optimizing 1 batches of size 50:
Time per Batch: 2.57
Time per Epoch: 2.83

Training Set: Loss: 2.883e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.486
Test Set: Loss: 0.000e+00, Accuracy: 0.458

Previous Layer Relative # of 0s: 0.6321782

            Ep

Time per Epoch: 2.78

Training Set: Loss: 2.883e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.486
Test Set: Loss: 0.000e+00, Accuracy: 0.458

Previous Layer Relative # of 0s: 0.6321782

            Epoch 280            
Abalone_FC_L1_mhl15_hl100_rpt0001_nTOL4_eTOL4_b50_e400
Trainable Hidden Layer Index: 11
GPU: 1

Optimizing 1 batches of size 50:
Time per Batch: 2.53
Time per Epoch: 2.78

Training Set: Loss: 2.873e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.471
Test Set: Loss: 0.000e+00, Accuracy: 0.422

Previous Layer Relative # of 0s: 0.6321782

            Epoch 281            
Abalone_FC_L1_mhl15_hl100_rpt0001_nTOL4_eTOL4_b50_e400
Trainable Hidden Layer Index: 11
GPU: 1

Optimizing 1 batches of size 50:
Time per Batch: 2.53
Time per Epoch: 2.78

Training Set: Loss: 2.883e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.486
Test Set: Loss: 0.000e+00, Accuracy: 0.458

Previous Layer Relative # of 0s: 0.6321782

            Ep

Time per Epoch: 2.78

Training Set: Loss: 2.882e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.486
Test Set: Loss: 0.000e+00, Accuracy: 0.457

Previous Layer Relative # of 0s: 0.6321782

            Epoch 298            
Abalone_FC_L1_mhl15_hl100_rpt0001_nTOL4_eTOL4_b50_e400
Trainable Hidden Layer Index: 11
GPU: 1

Optimizing 1 batches of size 50:
Time per Batch: 2.56
Time per Epoch: 2.82

Training Set: Loss: 2.872e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.471
Test Set: Loss: 0.000e+00, Accuracy: 0.422

Previous Layer Relative # of 0s: 0.6321782

            Epoch 299            
Abalone_FC_L1_mhl15_hl100_rpt0001_nTOL4_eTOL4_b50_e400
Trainable Hidden Layer Index: 11
GPU: 1

Optimizing 1 batches of size 50:
Time per Batch: 2.52
Time per Epoch: 2.78

Training Set: Loss: 2.882e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.486
Test Set: Loss: 0.000e+00, Accuracy: 0.457

Previous Layer Relative # of 0s: 0.6321782

            Ep